Epoch 50

In [4]:
# -*- coding: utf-8 -*-
"""resnet_overfitting_included.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1x-dq99w3wIjofB1NVMmbTqd-4Eh7O4OE
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 50
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Result/fruit_batch10_epoch50.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    #print(name)
    filename1 = os.fsdecode(name)
    #print(filename1)
    s=[]
    s= filename1.split('_')
    #print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        #print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                #print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img2= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

trainImages = np.array([i[0] for i in train_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])
# print(trainLabels[5])
#print(trainLabels)
trainLabels = to_categorical(trainLabels)
# #trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
# print('now')
# print(trainLabels[5])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')

testImages = np.array([i[0] for i in test_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])
#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x





def resnet_v2(input_shape, depth, num_classes):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# if version == 2:
#     model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# else:
#     model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Model'
model_name = 'Resnet_model_fNet_batch10_epoch50.h5'
saved_model= save_dir + '/' + model_name
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)




model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.2, shuffle=True, callbacks=[es, mc])


saved_model_final = load_model(saved_model)

# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()

Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 224, 224, 16)         448       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_20 (Ba  (None, 224, 224, 16)         64        ['conv2d_26[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_20 (Activation)  (None, 224, 224, 16)         0         ['

66/66 [==============================] - 146s 2s/step
[[8.4893503e-07 1.3724293e-12 0.0000000e+00 ... 2.6070281e-34
  1.9403239e-10 9.9999917e-01]
 [5.0499630e-15 3.1129332e-10 0.0000000e+00 ... 2.5237270e-35
  3.4786624e-13 1.0000000e+00]
 [9.8694978e-13 3.6583652e-04 5.8633735e-33 ... 4.2652381e-28
  4.5551898e-13 9.9963415e-01]
 ...
 [1.6418337e-33 3.7916079e-23 1.2026554e-23 ... 3.7622409e-25
  5.3447087e-27 0.0000000e+00]
 [9.7111206e-26 1.2307850e-13 0.0000000e+00 ... 1.7927700e-25
  2.8672202e-11 1.0000000e+00]
 [2.4003631e-26 4.3397381e-14 0.0000000e+00 ... 8.7049100e-37
  1.6830968e-19 1.0000000e+00]]


Epoch 75

In [5]:
# -*- coding: utf-8 -*-
"""resnet_overfitting_included.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1x-dq99w3wIjofB1NVMmbTqd-4Eh7O4OE
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 75
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Result/fruit_batch10_epoch75.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    #print(name)
    filename1 = os.fsdecode(name)
    #print(filename1)
    s=[]
    s= filename1.split('_')
    #print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        #print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                #print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img2= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

trainImages = np.array([i[0] for i in train_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])
# print(trainLabels[5])
#print(trainLabels)
trainLabels = to_categorical(trainLabels)
# #trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
# print('now')
# print(trainLabels[5])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')

testImages = np.array([i[0] for i in test_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])
#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x





def resnet_v2(input_shape, depth, num_classes):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# if version == 2:
#     model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# else:
#     model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Model'
model_name = 'Resnet_model_fNet_batch10_epoch75.h5'
saved_model= save_dir + '/' + model_name
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)




model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.2, shuffle=True, callbacks=[es, mc])


saved_model_final = load_model(saved_model)

# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()

Learning rate:  0.001
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_39 (Conv2D)          (None, 224, 224, 16)         448       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_30 (Ba  (None, 224, 224, 16)         64        ['conv2d_39[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_30 (Activation)  (None, 224, 224, 16)         0         ['

66/66 [==============================] - 201s 3s/step
[[2.30880493e-09 1.33385469e-10 2.50976196e-24 ... 4.03891261e-25
  4.00505269e-17 3.13981550e-24]
 [1.38226230e-08 1.91164862e-10 1.36556225e-02 ... 1.69580051e-12
  7.80309435e-08 4.67259611e-07]
 [9.99996066e-01 1.73108710e-08 4.35196936e-19 ... 1.10636875e-27
  3.72029990e-06 1.19064874e-17]
 ...
 [1.22776623e-06 4.29860108e-07 8.25177565e-33 ... 6.68998914e-26
  9.99998331e-01 6.40239382e-16]
 [2.54371946e-09 1.38717347e-12 9.71598956e-23 ... 0.00000000e+00
  1.00000000e+00 6.99166257e-13]
 [1.64281634e-13 1.44278274e-12 4.13234353e-08 ... 6.76243904e-37
  9.99998808e-01 1.25385308e-16]]


Epoch 100

In [ ]:
# -*- coding: utf-8 -*-
"""resnet_overfitting_included.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1x-dq99w3wIjofB1NVMmbTqd-4Eh7O4OE
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Result/fruit_batch10_epoch100.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    #print(name)
    filename1 = os.fsdecode(name)
    #print(filename1)
    s=[]
    s= filename1.split('_')
    #print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        #print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                #print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img2= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

trainImages = np.array([i[0] for i in train_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])
# print(trainLabels[5])
#print(trainLabels)
trainLabels = to_categorical(trainLabels)
# #trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
# print('now')
# print(trainLabels[5])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')

testImages = np.array([i[0] for i in test_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])
#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x





def resnet_v2(input_shape, depth, num_classes):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# if version == 2:
#     model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# else:
#     model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Model'
model_name = 'Resnet_model_fNet_batch10_epoch100.h5'
saved_model= save_dir + '/' + model_name
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)




model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.2, shuffle=True, callbacks=[es, mc])


saved_model_final = load_model(saved_model)

# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()

Epoch 150

In [6]:
# -*- coding: utf-8 -*-
"""resnet_overfitting_included.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1x-dq99w3wIjofB1NVMmbTqd-4Eh7O4OE
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 150
data_augmentation = False
num_classes = 12
test_img_num = 2363
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Result/fruit_batch10_epoch150.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    #print(name)
    filename1 = os.fsdecode(name)
    #print(filename1)
    s=[]
    s= filename1.split('_')
    #print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        #print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                #print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img2= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

trainImages = np.array([i[0] for i in train_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])
# print(trainLabels[5])
#print(trainLabels)
trainLabels = to_categorical(trainLabels)
# #trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
# print('now')
# print(trainLabels[5])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')

testImages = np.array([i[0] for i in test_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])
#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x





def resnet_v2(input_shape, depth, num_classes):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# if version == 2:
#     model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# else:
#     model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Model'
model_name = 'Resnet_model_fNet_6fruit_batch10_epoch150.h5'
saved_model= save_dir + '/' + model_name
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)




model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.2, shuffle=True, callbacks=[es, mc])


saved_model_final = load_model(saved_model)

# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()

Learning rate:  0.001
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_52 (Conv2D)          (None, 224, 224, 16)         448       ['input_5[0][0]']             
                                                                                                  
 batch_normalization_40 (Ba  (None, 224, 224, 16)         64        ['conv2d_52[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_40 (Activation)  (None, 224, 224, 16)         0         ['

Epoch : 200

In [ ]:
# -*- coding: utf-8 -*-
"""resnet_overfitting_included.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1x-dq99w3wIjofB1NVMmbTqd-4Eh7O4OE
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Result/fruit_batch10_epoch200.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    #print(name)
    filename1 = os.fsdecode(name)
    #print(filename1)
    s=[]
    s= filename1.split('_')
    #print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        #print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                #print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img2= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

trainImages = np.array([i[0] for i in train_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])
# print(trainLabels[5])
#print(trainLabels)
trainLabels = to_categorical(trainLabels)
# #trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
# print('now')
# print(trainLabels[5])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')

testImages = np.array([i[0] for i in test_data])#.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])
#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x





def resnet_v2(input_shape, depth, num_classes):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# if version == 2:
#     model = resnet_v2(input_shape=input_shape, depth=depth, num_classes= num_classes)
# else:
#     model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Resnet_Model'
model_name = 'Resnet_model_fNet_batch10_epoch200.h5'
saved_model= save_dir + '/' + model_name
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)




model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.2, shuffle=True, callbacks=[es, mc])


saved_model_final = load_model(saved_model)

# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()